# Settings

In [ ]:
# import shahlabdata.wgs
import cn_utils

# Data

In [4]:
isabl_cohort = 'HCMI GDAN AWG'
cn_data = shahlabdata.wgs.get_cohort_cn(isabl_cohort, most_recent=True)

Retrieving 1055 from analyses API endpoint...
retrieved 500 out of 1055...
retrieved 1000 out of 1055...
retrieved 1055 out of 1055...


In [25]:
df = cn_data[['isabl_sample_id']]

In [26]:
df['project'] = [s.split('-')[1] for s in df['isabl_sample_id']]

/juno/work/shah/users/chois7/envs/p37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
df = df.drop_duplicates().reset_index(drop=True)

In [30]:
df.head()

,isabl_sample_id,project
0,HCM-WCMC-0673-C56-01A,WCMC
1,HCM-WCMC-0673-C56-05A,WCMC
2,HCM-WCMC-0673-C56-85A,WCMC
3,HCM-WCMC-0673-C56-85B,WCMC
4,HCM-BROD-0476-C56-85M,BROD


In [29]:
tumor_type_path = '/juno/work/shah/users/chois7/tickets/hcmi/metadata/tumor_types.tsv'
df.to_csv(tumor_type_path, sep='\t', index=False)

# Run

In [ ]:
update_regions = False
logging.basicConfig(level = logging.DEBUG)
# gene_list_path = '/juno/work/shah/users/chois7/tickets/cohort-cn-qc/resources/gene_list.txt'
gene_list_path = '/juno/work/shah/users/chois7/tickets/cohort-cn-qc/resources/gene_list.simple.txt'
# for cohorts in (['SPECTRUM-DLP'], ['SPECTRUM'], ['Metacohort'], ['APOLLO-H'], ['SPECTRUM', 'Metacohort', 'APOLLO-H']):
# for cohorts in (['APOLLO-H'], ['SPECTRUM', 'Metacohort', 'APOLLO-H']):
#for cohorts in (['SPECTRUM-DLP'], ['SPECTRUM'], ['Metacohort']):

In [ ]:
for cohorts in (['HCMI'],):
    cn = CopyNumberChangeData(gene_list=gene_list_path, cohorts=cohorts, update_regions=update_regions)
    plot_peaks_troughs(cn, cn.peak_params)
    plot_merged_peaks_troughs(cn.peaks_troughs)
    # break

    cohort_symbol = '_'.join(cn.cohorts)
    # break
    for signature in cn.signature_counts:
        logging.info(f'processing cohorts:{cohorts} tumor-code:{signature}')
        if cn.signature_counts[signature] > 3:
            logging.info(f'plotting cohorts:{cohorts} tumor-code:{signature}')
            cn.plot_pan_chrom_cn(group=signature, out_path=f'hcmi/results/{cohort_symbol}.{signature}.pdf')
            cn.plot_per_chrom_cn(group=signature, out_path=f'hcmi/results/{cohort_symbol}.{signature}.per-chrom.pdf')

    if not update_regions:
        cn.gene_ranges.update(cn.peaks_troughs)
    gene_cn = cn.get_gene_cn_counts()
    results = evaluate_enrichment(cn.signatures, cn.signature_counts, cn.gene_ranges.keys(),
            cn.sample_counts, padj_cutoff=0.1)
    results.to_csv(f'hcmi/results/enrichment/enrichment.{cohort_symbol}.tsv', sep='\t', index=False)